# <font color='teal'>North Carolina County Demographics and Election Results Dashboard</font>
---
Usage:
> <b><font color='#000000'>Click Cells > Run All</font></b> in the toolbar above. The dashboard will load in 5-10 seconds below this text.
> 
> This project contains a dashboard allowing the user to analyze county-level voting results alongside demographic trends. Use the inputs to specify the data you want to return, then click <i>Update</i> to generate maps, charts, and tables. These demonstrate county level voting patterns and demographic trends. Counties voting more Democratic are shown in blue and Republican are shown in red. Hover over the map uncover demographic data. The bar charts showcases the largest counties in the state (having 100,000+ residents). In the <i>Table</i> tab, use the controls to filter the tables in place.
> 
> <i>All data is downloadable and verifiable via the data source links below.</i>
>
Data Sources:
> + [North Carolina State Board of Elections (NCSBE)](https://www.ncsbe.gov/results-data/election-results/historical-election-results-data#by-precinct)
> + [American Community Survey (ACS)](https://www.census.gov/programs-surveys/acs)
>
Author: Dave Russell
> + [Github](https://github.com/drussel4/)
> + [Blog](https://datadave.substack.com/)
> + davidjeffreyrussell@gmail.com

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import os
import time
import requests
import json
import ipywidgets as ipw
from urllib.request import urlopen
import urllib.request

# Graphing libraries
import plotly.express as px
from IPython.display import display

In [ ]:
# Set pandas formatting for comma as thousands sep
pd.options.display.float_format = '{:,}'.format

In [ ]:
# Store your API key as a variable
census_gov_key = open('misc/census_gov_key.txt', 'r').read()

In [ ]:
# Set location for downloads form app
app_download_path = 'Jupyter Downloads'

In [ ]:
# Downloaded precinct level result data txt files from:
# https://www.ncsbe.gov/results-data/election-results/historical-election-results-data#by-precinct
# Retrieve and store as DataFrames in dict
nc_pct_results_folder = 'Election Results'

In [ ]:
# Walk os to retrieve files from folder
onlyfiles = [f for f in os.listdir(nc_pct_results_folder) if os.path.isfile(os.path.join(nc_pct_results_folder, f))]
full_paths_list = []
for f in onlyfiles:
    full_paths_list.append(nc_pct_results_folder + '\\' + f)

In [ ]:
# Loop through each year's file, beginning with 2014
yearly_dfs_dict = {}
years = []
for file in full_paths_list:
    
    year = file.split('results_pct_')[1][0:4]
    years.append(int(year))
    
    # Read table from txt file
    year_results_df = pd.read_csv(file,
                                  sep='\t',
                                  error_bad_lines=False)
    
    # Drop columns that were unnamed
    cols_list = list(year_results_df.columns)
    bad_cols_list = []
    for col in cols_list:
        if 'Unnamed' in col or 'Real Precinct' in col:
            bad_cols_list.append(col)
    year_results_df.drop(bad_cols_list, axis=1, inplace=True)
    
    # Drop parties other than D/R
    year_results_df = year_results_df[year_results_df['Choice Party'].isin(['DEM', 'REP'])]
    yearly_dfs_dict[int(year)] = year_results_df
#     print('Extracted {} from txt file'.format(year))
    

In [ ]:
# Clean data for each year's df
filtered_dfs_dict = {}
for year in years:
    
    year_results_df = yearly_dfs_dict[year]
    # Retrieve unique contests
    unique_contests = list(year_results_df['Contest Name'].unique())

    # Create list of contests to retain upon filter
    contests_to_examine = ['US PRESIDENT']
    for contest in unique_contests:
        if 'US HOUSE OF REPRESENTATIVES' in contest\
        or 'US SENATE' in contest\
        or 'NC STATE SENATE' in contest\
        or 'NC HOUSE OF REPRESENTATIVES' in contest:
            contests_to_examine.append(contest)

    # Reduce DataFrame to only contests of interest
    year_results_df = year_results_df[year_results_df['Contest Name'].isin(
        contests_to_examine)]
    
    # Aggregate results to county level from precinct
    year_agg_results_df = year_results_df.groupby(
        ['Contest Name', 'County', 'Choice Party'], as_index=False).sum()
    
    # Create Series with generic tag for Senate and House races
    contest_names_list = list(year_agg_results_df['Contest Name'])
    contest_generics_list = []
    for c in contest_names_list:
        if 'US HOUSE' in c:
            contest_generics_list.append('US House')
        elif 'US SENATE' in c:
            contest_generics_list.append('US Senate')
        elif 'US PRESIDENT' in c:
            contest_generics_list.append('President')
        elif 'NC STATE SENATE' in c:
            contest_generics_list.append('NC Senate')
        elif 'NC HOUSE OF REPRESENTATIVES' in c:
            contest_generics_list.append('NC House')
        else:
            contest_generics_list.append('')
    year_agg_results_df['Contest Generic'] = contest_generics_list
    
    # Aggregate vote counts to generic contest levels
    year_generic_results_df = year_agg_results_df.groupby(
        ['Contest Generic', 'County', 'Choice Party'], as_index=False).sum()

    # Calculate 'Early' count from 'One Stop, 'Absentee by Mail', and 'Provisional'
    year_generic_results_df['Early'] = (
        year_generic_results_df['One Stop']
        + year_generic_results_df['Absentee by Mail']
        + year_generic_results_df['Provisional'])
    
    filtered_dfs_dict[year] = year_generic_results_df
#     print('Cleaned {} DataFrame'.format(year))
    

In [ ]:
# County results dicts for each year
county_dfs_dict = {}
for year in years:
    
    year_generic_results_df = filtered_dfs_dict[year]
    counties_list = list(year_generic_results_df['County'].unique())
    
    # County results dict
    total_results_dict = {}
    early_results_dict = {}
    eday_results_dict = {}
    if year in [2016, 2020]:
        race_types = ['President', 'US Senate', 'US House', 'NC Senate', 'NC House']
    elif year in [2018]:
        race_types = ['US House', 'NC Senate', 'NC House']
    else:
        race_types = ['US Senate', 'US House', 'NC Senate', 'NC House']
    for race in race_types:
        county_total_dict = {}
        county_early_dict = {}
        county_eday_dict = {}

        # For each county, create DataFrame and identify winning party, margins
        for county in counties_list:
            temp_df = year_generic_results_df[(
                year_generic_results_df['Contest Generic'] == race) & (
                year_generic_results_df['County'] == county)].reset_index()
            winner_index = temp_df['Total Votes'].idxmax()
            loser_index = temp_df['Total Votes'].idxmin()
            temp_df['Total % of Vote'] = round(
                temp_df['Total Votes']
                / temp_df['Total Votes'].sum() * 100, 1)
            temp_df['Early % of Vote'] = round(
                temp_df['Early']
                / temp_df['Early'].sum() * 100, 1)
            temp_df['Election Day % of Vote'] = round(
                temp_df['Election Day']
                / temp_df['Election Day'].sum() * 100, 1)
            if 'DEM' in list(temp_df['Choice Party']):
                dem_index = temp_df.index[temp_df['Choice Party'] == 'DEM'].tolist()[0]
            else:
                dem_index = None
            if 'REP' in list(temp_df['Choice Party']):
                rep_index = temp_df.index[temp_df['Choice Party'] == 'REP'].tolist()[0]
            else:
                rep_index = None
            winner_party = temp_df.iloc[winner_index]['Choice Party']
            
            # Where both parties had a candidate, grab margins and votes cast
            if dem_index is not None and rep_index is not None:
                
                # Total
                tot_votes_cast = float((temp_df.iloc[dem_index]['Total Votes']
                                  + temp_df.iloc[rep_index]['Total Votes']))
                total_margin = float((temp_df.iloc[dem_index]['Total Votes']
                                - temp_df.iloc[rep_index]['Total Votes']))
                total_percent = float(round((temp_df.iloc[dem_index]['Total % of Vote']
                                       - temp_df.iloc[rep_index]['Total % of Vote']), 1))

                # Election Day
                eday_votes_cast = float((temp_df.iloc[dem_index]['Election Day']
                                   + temp_df.iloc[rep_index]['Election Day']))
                eday_margin = float((temp_df.iloc[dem_index]['Election Day']
                               - temp_df.iloc[rep_index]['Election Day']))
                eday_percent = float(round((temp_df.iloc[dem_index]['Election Day % of Vote']
                                      - temp_df.iloc[rep_index]['Election Day % of Vote']), 1))

                # Early (Absentee, One-Stop, and Provisional)
                early_votes_cast = float((temp_df.iloc[dem_index]['Early']
                                    + temp_df.iloc[rep_index]['Early']))
                early_margin = float((temp_df.iloc[dem_index]['Early']
                                - temp_df.iloc[rep_index]['Early']))
                early_percent = float(round((temp_df.iloc[dem_index]['Early % of Vote']
                                       - temp_df.iloc[rep_index]['Early % of Vote']), 1))
            
            # Where Democrat ran unnoposed, derive margins and grab votes cast
            elif rep_index is None:
                
                # Total
                tot_votes_cast = float(temp_df.iloc[dem_index]['Total Votes'])
                total_margin = float(temp_df.iloc[dem_index]['Total Votes'])
                total_percent = 100.0

                # Election Day
                eday_votes_cast = float(temp_df.iloc[dem_index]['Election Day'])
                eday_margin = float(temp_df.iloc[dem_index]['Election Day'])
                eday_percent = 100.0

                # Early (Absentee, One-Stop, and Provisional)
                early_votes_cast = float(temp_df.iloc[dem_index]['Early'])
                early_margin = float(temp_df.iloc[dem_index]['Early'])
                early_percent = 100.0
            
            # Where Republican ran unnoposed, derive margins and grab votes cast
            elif dem_index is None:
                
                # Total
                tot_votes_cast = float(temp_df.iloc[rep_index]['Total Votes'])
                total_margin = float(temp_df.iloc[rep_index]['Total Votes'])
                total_percent = -100.0

                # Election Day
                eday_votes_cast = float(temp_df.iloc[rep_index]['Election Day'])
                eday_margin = float(temp_df.iloc[rep_index]['Election Day'])
                eday_percent = -100.0

                # Early (Absentee, One-Stop, and Provisional)
                early_votes_cast = float(temp_df.iloc[rep_index]['Early'])
                early_margin = float(temp_df.iloc[rep_index]['Early'])
                early_percent = -100.0
            
            else:
                print('Error identifying party')
                print(year, '|', race, '|', county)

            # Assign to dicts
            county_total_dict[county] = {'Winning Party':winner_party,
                                         'Margin Count':total_margin,
                                         'Margin Percent':total_percent,
                                         'Votes Cast':tot_votes_cast}
            county_eday_dict[county] = {'Winning Party':winner_party,
                                        'Margin Count':eday_margin,
                                        'Margin Percent':eday_percent,
                                        'Votes Cast':eday_votes_cast}
            county_early_dict[county] = {'Winning Party':winner_party,
                                         'Margin Count':early_margin,
                                         'Margin Percent':early_percent,
                                         'Votes Cast':early_votes_cast}

        total_results_dict[race] = county_total_dict
        early_results_dict[race] = county_early_dict
        eday_results_dict[race] = county_eday_dict
    
    county_dfs_dict[year] = {'Total':total_results_dict,
                             'Early':early_results_dict,
                             'Election Day':eday_results_dict}
#     print('Calculated {} data'.format(year))
    

In [ ]:
# County results dfs for each year
yearly_results_dfs_dict = {}
for year in years:
    
    # Make DataFrames for each race and voting method
    
    # President
    if year in [2016, 2020]:
        pres_total_df = pd.DataFrame.from_dict(
            county_dfs_dict[year]['Total']['President'], orient='index')
        pres_early_df = pd.DataFrame.from_dict(
            county_dfs_dict[year]['Early']['President'], orient='index')
        pres_eday_df = pd.DataFrame.from_dict(
            county_dfs_dict[year]['Election Day']['President'], orient='index')
    else:
        pres_total_df = pd.DataFrame()
        pres_early_df = pd.DataFrame()
        pres_eday_df = pd.DataFrame()

    # US Senate
    if year in [2014, 2016, 2020]:
        us_sen_total_df = pd.DataFrame.from_dict(
            county_dfs_dict[year]['Total']['US Senate'], orient='index')
        us_sen_early_df = pd.DataFrame.from_dict(
            county_dfs_dict[year]['Early']['US Senate'], orient='index')
        us_sen_eday_df = pd.DataFrame.from_dict(
            county_dfs_dict[year]['Election Day']['US Senate'], orient='index')
    else:
        us_sen_total_df = pd.DataFrame()
        us_sen_early_df = pd.DataFrame()
        us_sen_eday_df = pd.DataFrame()

    # US House
    if year in [2014, 2016, 2018, 2020]:
        us_house_total_df = pd.DataFrame.from_dict(
            county_dfs_dict[year]['Total']['US House'], orient='index')
        us_house_early_df = pd.DataFrame.from_dict(
            county_dfs_dict[year]['Early']['US House'], orient='index')
        us_house_eday_df = pd.DataFrame.from_dict(
            county_dfs_dict[year]['Election Day']['US House'], orient='index')
    else:
        us_house_total_df = pd.DataFrame()
        us_house_early_df = pd.DataFrame()
        us_house_eday_df = pd.DataFrame()
    
    # NC Senate
    if year in [2014, 2016, 2018, 2020]:
        nc_sen_total_df = pd.DataFrame.from_dict(
            county_dfs_dict[year]['Total']['NC Senate'], orient='index')
        nc_sen_early_df = pd.DataFrame.from_dict(
            county_dfs_dict[year]['Early']['NC Senate'], orient='index')
        nc_sen_eday_df = pd.DataFrame.from_dict(
            county_dfs_dict[year]['Election Day']['NC Senate'], orient='index')
    else:
        nc_sen_total_df = pd.DataFrame()
        nc_sen_early_df = pd.DataFrame()
        nc_sen_eday_df = pd.DataFrame()
    
    # NC House
    if year in [2014, 2016, 2018, 2020]:
        nc_house_total_df = pd.DataFrame.from_dict(
            county_dfs_dict[year]['Total']['NC House'], orient='index')
        nc_house_early_df = pd.DataFrame.from_dict(
            county_dfs_dict[year]['Early']['NC House'], orient='index')
        nc_house_eday_df = pd.DataFrame.from_dict(
            county_dfs_dict[year]['Election Day']['NC House'], orient='index')
    else:
        nc_house_total_df = pd.DataFrame()
        nc_house_early_df = pd.DataFrame()
        nc_house_eday_df = pd.DataFrame()
    
    # Store in dict
    result_dfs_dict = {
        'President':{
            'Total':pres_total_df,
            'Early':pres_early_df,
            'Election Day':pres_eday_df},
        'US Senate':{
            'Total':us_sen_total_df,
            'Early':us_sen_early_df,
            'Election Day':us_sen_eday_df},
        'US House':{
            'Total':us_house_total_df,
            'Early':us_house_early_df,
            'Election Day':us_house_eday_df},
        'NC Senate':{
            'Total':nc_sen_total_df,
            'Early':nc_sen_early_df,
            'Election Day':nc_sen_eday_df},
        'NC House':{
            'Total':nc_house_total_df,
            'Early':nc_house_early_df,
            'Election Day':nc_house_eday_df}}
    
    yearly_results_dfs_dict[year] = result_dfs_dict
#     print('Standardized {} data'.format(year))
    

In [ ]:
# Metrics to query from ACS database
acs_metrics_dict = {
    'K200101_001E':'Population',
    'K200101_002E':'Pop %, Male',
    'K200101_003E':'Pop %, Female',
    'K200104_002E':'Pop %, Age <18',
    'K200104_003E':'Pop %, Age 18-24',
    'K200104_004E':'Pop %, Age 25-34',
    'K200104_005E':'Pop %, Age 35-44',
    'K200104_006E':'Pop %, Age 45-54',
    'K200104_007E':'Pop %, Age 55-64',
    'K200104_008E':'Pop %, Age 65+',
    'K200201_002E':'Pop %, White',
    'K200201_003E':'Pop %, Black',
    'K200201_004E':'Pop %, N. American & Alaskan',
    'K200201_005E':'Pop %, Asian',
    'K200201_006E':'Pop %, Hawaiian & P. Islander',
    'K200201_007E':'Pop %, Other',
    'K200201_008E':'Pop %, 2+ Races',
    'K201501_003E':'Pop %, 9-12th Grade',
    'K201501_004E':'Pop %, High School',
    'K201501_005E':'Pop %, Some College No Degree',
    'K201501_006E':'Pop %, Assoc Degree',
    'K201501_007E':'Pop %, Bach Degree',
    'K201501_008E':'Pop %, Graduate Degree',
    'K201902_001E':'Median Household Income',
    'K202702_002E':'Pop %, Priv Health Ins',
    'K202702_003E':'Pop %, No Priv Health Ins',
    'K202703_002E':'Pop %, Publ Health Ins',
    'K202703_003E':'Pop %, No Publ Health Ins'}

acs_metrics_dtypes = {
    'K200101_001E':'float',
    'K200101_002E':'float',
    'K200101_003E':'float',
    'K200104_002E':'float',
    'K200104_003E':'float',
    'K200104_004E':'float',
    'K200104_005E':'float',
    'K200104_006E':'float',
    'K200104_007E':'float',
    'K200104_008E':'float',
    'K200201_002E':'float',
    'K200201_003E':'float',
    'K200201_004E':'float',
    'K200201_005E':'float',
    'K200201_006E':'float',
    'K200201_007E':'float',
    'K200201_008E':'float',
    'K201501_003E':'float',
    'K201501_004E':'float',
    'K201501_005E':'float',
    'K201501_006E':'float',
    'K201501_007E':'float',
    'K201501_008E':'float',
    'K201902_001E':'float',
    'K202702_002E':'float',
    'K202702_003E':'float',
    'K202703_002E':'float',
    'K202703_003E':'float'}

acs_metrics_list = list(acs_metrics_dict.keys())
acs_metrics_str = ''
for metric in acs_metrics_list:
    acs_metrics_str = acs_metrics_str + metric + ','
acs_metrics_str = acs_metrics_str[:-1] # Drop last comma

In [ ]:
# Retrieve North Carolina county FIPS and populations
yearly_merged_dfs_dict = {}
state = 37 # 37 is North Carolina
for year in years:
    
    # Query American Community Survey (ACS) API
    # Set year compatible with ACS API (as of Jan 2021, 2020 data not yet available, must use 2019)
    if year == 2020:
        data_url = 'https://api.census.gov/data/2019/acs/acsse?get=NAME,{}&for=county:*&in=state:{}&key={}'.format(
            acs_metrics_str, state, census_gov_key)
    else:
        data_url = 'https://api.census.gov/data/{}/acs/acsse?get=NAME,{}&for=county:*&in=state:{}&key={}'.format(
            year, acs_metrics_str, state, census_gov_key)
    response = requests.get(data_url)
    data = response.json()
    acs_orig_df = pd.DataFrame(data[1:], columns=data[0])
    
    # Fill null float values with numpy NaNs
    # Separate DataFrame into two, one with numerics and one without
    acs_floats_df = acs_orig_df[acs_metrics_list]
    acs_non_floats_df = acs_orig_df[acs_orig_df.columns.difference(acs_metrics_list)]
    acs_floats_df[acs_metrics_list].fillna(value=np.nan, inplace=True)
    acs_df = pd.concat([acs_non_floats_df, acs_floats_df], axis=1)
    
    # Retrieve short name, convert to dict
    acs_df['county_name'] = acs_df['NAME'].str.split(' County', 0, expand=True)[0].str.upper()
    acs_df = acs_df.astype(acs_metrics_dtypes).copy()
    acs_df = acs_df.rename(columns=acs_metrics_dict)
    acs_df['FIPS'] = acs_df['state'] + acs_df['county']
    acs_df.set_index('county_name', inplace=True)
    acs_df.drop(columns=['state', 'county', 'NAME'], inplace=True)
    
    # Calculate % of pop for each ACS metric
    for acs_metric in acs_metrics_dict.values():
        if acs_metric not in ['Population', 'Median Household Income']:
            acs_df[acs_metric] = round(acs_df[acs_metric] / acs_df['Population'] * 100, 1)
    
    if year in [2016, 2020]:
        race_types = ['President', 'US Senate', 'US House', 'NC Senate', 'NC House']
        
        merged_dfs_dict = {
            'President':{
                'Total':pd.DataFrame(),
                'Early':pd.DataFrame(),
                'Election Day':pd.DataFrame()},
            'US Senate':{
                'Total':pd.DataFrame(),
                'Early':pd.DataFrame(),
                'Election Day':pd.DataFrame()},
            'US House':{
                'Total':pd.DataFrame(),
                'Early':pd.DataFrame(),
                'Election Day':pd.DataFrame()},
            'NC Senate':{
                'Total':pd.DataFrame(),
                'Early':pd.DataFrame(),
                'Election Day':pd.DataFrame()},
            'NC House':{
                'Total':pd.DataFrame(),
                'Early':pd.DataFrame(),
                'Election Day':pd.DataFrame()}}
    elif year in [2018]:
        race_types = ['US House', 'NC Senate', 'NC House']
        merged_dfs_dict = {
            'US House':{
                'Total':pd.DataFrame(),
                'Early':pd.DataFrame(),
                'Election Day':pd.DataFrame()},
            'NC Senate':{
                'Total':pd.DataFrame(),
                'Early':pd.DataFrame(),
                'Election Day':pd.DataFrame()},
            'NC House':{
                'Total':pd.DataFrame(),
                'Early':pd.DataFrame(),
                'Election Day':pd.DataFrame()}}
    else: # Implicitly all years (2014, 2016, 2018, 2020)
        race_types = ['US Senate', 'US House', 'NC Senate', 'NC House']
        merged_dfs_dict = {
            'US Senate':{
                'Total':pd.DataFrame(),
                'Early':pd.DataFrame(),
                'Election Day':pd.DataFrame()},
            'US House':{
                'Total':pd.DataFrame(),
                'Early':pd.DataFrame(),
                'Election Day':pd.DataFrame()},
            'NC Senate':{
                'Total':pd.DataFrame(),
                'Early':pd.DataFrame(),
                'Election Day':pd.DataFrame()},
            'NC House':{
                'Total':pd.DataFrame(),
                'Early':pd.DataFrame(),
                'Election Day':pd.DataFrame()}}
        
    for race in race_types:
        for vote_method in ['Total', 'Early', 'Election Day']:
         
            # Merge ACS data to election results
            # Right merge because ACS data not avail for all 100 counties
            merged_df = acs_df.merge(
                yearly_results_dfs_dict[year][race][vote_method],
                how='right', left_index=True, right_index=True)
            merged_df.sort_index(ascending=True, inplace=True)
            merged_df['Votes % of Pop'] = round(
                merged_df['Votes Cast'] / merged_df['Population'] * 100, 1)
            merged_dfs_dict[race][vote_method] = merged_df
            
    yearly_merged_dfs_dict[year] = merged_dfs_dict
#     print('Merged {} election and ACS data'.format(year))

In [ ]:
# Concatenate master df comprising all years
master_columns = {}
for col in ['County'] + list(
    yearly_merged_dfs_dict[2020]['President']['Total'].columns) + [
    'Year', 'Contest', 'Method']:
    master_columns[col] = [None]
master_df = pd.DataFrame(data=master_columns)

for year in years:
    for race_key in yearly_merged_dfs_dict[year].keys():
        for method_key in yearly_merged_dfs_dict[year][race_key].keys():
            temp_df = yearly_merged_dfs_dict[year][race_key][method_key]
            temp_df = temp_df.reset_index().rename(columns={'index':'County'})
            temp_df['Year'] = int(year)
            temp_df['Contest'] = race_key
            temp_df['Method'] = method_key
            master_df = pd.concat([master_df, temp_df], axis=0)
            
#     print('Consolidated {} to master_df'.format(year))

# Explicitly drop first row, which was Nulls in original df creation
master_df.drop(index=0, inplace=True)

# Reorder columns
master_df = master_df[
    ['County', 'FIPS', 'Year', 'Contest', 'Method', 'Winning Party',
     'Margin Count', 'Margin Percent', 'Votes Cast', 'Votes % of Pop',
     'Population', 'Pop %, Male', 'Pop %, Female', 'Pop %, Age <18',
     'Pop %, Age 18-24', 'Pop %, Age 25-34', 'Pop %, Age 35-44',
     'Pop %, Age 45-54', 'Pop %, Age 55-64', 'Pop %, Age 65+',
     'Pop %, White', 'Pop %, Black', 'Pop %, N. American & Alaskan',
     'Pop %, Asian', 'Pop %, Hawaiian & P. Islander', 'Pop %, Other',
     'Pop %, 2+ Races', 'Pop %, 9-12th Grade', 'Pop %, High School',
     'Pop %, Some College No Degree', 'Pop %, Assoc Degree',
     'Pop %, Bach Degree', 'Pop %, Graduate Degree',
     'Median Household Income', 'Pop %, Priv Health Ins',
     'Pop %, No Priv Health Ins', 'Pop %, Publ Health Ins',
     'Pop %, No Publ Health Ins']]

In [ ]:
# Set styles and layouts
style_width_initial = {'description_width':'initial'}
style_width_max_content = {'width':'max-content'}
layout_width_200px = ipw.widgets.Layout(width='200px')
layout_width_300px = ipw.widgets.Layout(width='300px')

# Hidden/visible layouts
layout_hidden = ipw.widgets.Layout(visibility='hidden', width='200px')
layout_visible = ipw.widgets.Layout(visibility='visible', width='200px')
layout_no_width_hidden = ipw.widgets.Layout(visibility='hidden')
layout_no_width_visible = ipw.widgets.Layout(visibility='visible')
layout_small_table_filters_hidden = ipw.widgets.Layout(visibility='hidden', width='200px')
layout_small_table_filters_visible = ipw.widgets.Layout(visibility='visible', width='200px')
layout_large_table_filters_hidden = ipw.widgets.Layout(visibility='hidden', width='500px')
layout_large_table_filters_visible = ipw.widgets.Layout(visibility='visible', width='500px')
layout_tall_table_filters_hidden = ipw.widgets.Layout(visibility='hidden', width='200px', height='180px')
layout_tall_table_filters_visible = ipw.widgets.Layout(visibility='visible', width='200px', height='180px')

In [ ]:
# To calculate steps on int and float range sliders, apply smart step function
acceptable_steps = [1, 5, 10, 50, 100, 500, 1000, 10000, 50000, 100000]
def step_finder(acceptable_steps, target_series, num_steps=10):
    series_min = target_series.min()
    series_max = target_series.max()
    if series_min < 0:
        step = (series_max + abs(series_min)) / num_steps
        step_proximate = min(acceptable_steps, key=lambda x:abs(x-step))
    else:
        step = (series_max - series_min) / num_steps
        step_proximate = min(acceptable_steps, key=lambda x:abs(x-step))
    return(step_proximate)

In [ ]:
wgt_race_radio = ipw.widgets.RadioButtons(
    options=[('President', 'President'),
             ('US Senate', 'US Senate'),
             ('US House', 'US House'),
             ('NC Senate', 'NC Senate'),
             ('NC House', 'NC House')],
    value='President',
    description='Race:',
    style=style_width_initial,
    layout=layout_width_200px)

In [ ]:
wgt_method_radio = ipw.widgets.RadioButtons(
    options=[('Total Votes', 'Total'),
             ('Election Day Votes', 'Election Day'),
             ('Early Votes', 'Early')],
    value='Total',
    description='Voting Method:',
    style=style_width_initial,
    layout=layout_width_300px)

In [ ]:
wgt_calc_dropdown = ipw.widgets.Dropdown(
    options=[('Margin Percent', 'Margin Percent'),
             ('Margin Count', 'Margin Count'),
             ('Votes Cast', 'Votes Cast'),
             ('Votes % of Pop', 'Votes % of Pop')],
    value='Margin Percent',
    description='Metric:',
    style=style_width_initial,
    layout=layout_width_200px)

In [ ]:
wgt_app_desc = ipw.widgets.HTML(
    value='''
    <font color="#008080" font size=3><b>North Carolina County \
    Demographics and Election Results Dashboard</b></font>''',
    description='',
    style=style_width_initial)

In [ ]:
wgt_extra_space = ipw.widgets.HTML(
    value='''
    <font color="#FFFFFF">TKTK</font>''',
    description='',
    style=style_width_initial)

In [ ]:
wgt_help = ipw.widgets.HTML(
    value='',
    description='<font color="#FFA500"><b><i>Help</i></b></font>',
    description_tooltip='''
    Make selections and click Update to refresh the
exhibits. Click Download for a csv of the data.
ACS Overlay metrics are presented as % of population,
except for population itself.''',
    style=style_width_initial)

In [ ]:
wgt_sel_disp = ipw.widgets.HTML(
    value='<b>Selections: <font color="#008080">Race: {}, \
        Method: {}, Metric: {}, Overlay: {}</b></font>'.format(
        'President', 'Total', 'Margin Percent', 'Population'),
    description='',
    style=style_width_initial)

In [ ]:
wgt_update_button = ipw.widgets.Button(
    description='Update',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to refresh map with selections.',
    icon='refresh',
    style=style_width_max_content)

In [ ]:
wgt_update_status = ipw.widgets.HTML(
    value='',
    description='',
    tooltip='')

In [ ]:
wgt_update_progress = ipw.widgets.IntProgress(
    value=0,
    min=0,
    max=7,
    step=1,
    bar_style='info', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal',
    layout=layout_hidden)

In [ ]:
wgt_bar_header = ipw.widgets.HTML(
    value='<font color="#000000"><b><i>Bar Charts Contain Counties Having 100,000+ Residents</i></b></font>',
    description='',
    tooltip='',
    layout=layout_no_width_hidden)

In [ ]:
wgt_summary_table_header = ipw.widgets.HTML(
    value='<font color="#008080"><b><i>Yearly Summary Table</i></b></font>',
    description='',
    tooltip='',
    layout=layout_no_width_hidden)

In [ ]:
wgt_detailed_table_header = ipw.widgets.HTML(
    value='<font color="#008080"><b><i>County Detail Table</i></b></font>',
    description='',
    tooltip='',
    layout=layout_no_width_hidden)

In [ ]:
wgt_selmult_note = ipw.widgets.HTML(
    value='<font color="#008080"><b><i>Select 1+ Years and Columns</i></b></font>',
    description='',
    layout=layout_no_width_hidden)

In [ ]:
wgt_population_slider = ipw.widgets.IntRangeSlider(
    value=[acs_df['Population'].min(),
           acs_df['Population'].max()],
    min=acs_df['Population'].min(),
    max=acs_df['Population'].max(),
    step=step_finder(acceptable_steps,
                     acs_df['Population'],
                     num_steps=10),
    description='Population:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format=',d',
    style=style_width_initial,
    layout=layout_large_table_filters_hidden)

In [ ]:
wgt_year_selmult = ipw.widgets.SelectMultiple(
    options=[2014, 2016, 2018, 2020],
    value=[2020],
    description='Year(s)',
    style=style_width_initial,
    layout=layout_small_table_filters_hidden)

In [ ]:
wgt_winning_party_radio = ipw.widgets.RadioButtons(
    options=[('Both', ['DEM', 'REP']),
             ('Democrat', ['DEM']),
             ('Republican', ['REP'])],
    value=['DEM', 'REP'],
    description='Winning Party:',
    style=style_width_initial,
    layout=layout_small_table_filters_hidden)

In [ ]:
wgt_margin_count_slider = ipw.widgets.IntRangeSlider(
    value=[master_df['Margin Count'].min(), master_df['Margin Count'].max()],
    min=master_df['Margin Count'].min(),
    max=master_df['Margin Count'].max(),
    step=step_finder(acceptable_steps,
                     master_df['Margin Count'],
                     num_steps=10),
    description='Margin Count:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format=',d',
    style=style_width_initial,
    layout=layout_large_table_filters_hidden)

In [ ]:
wgt_margin_percent_slider = ipw.widgets.FloatRangeSlider(
    value=[master_df['Margin Percent'].min(),
           master_df['Margin Percent'].max()],
    min=master_df['Margin Percent'].min(),
    max=master_df['Margin Percent'].max(),
    step=step_finder(acceptable_steps,
                     master_df['Margin Percent'],
                     num_steps=10),
    description='Margin Percent:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format=',d',
    style=style_width_initial,
    layout=layout_large_table_filters_hidden)

In [ ]:
wgt_votes_cast_slider = ipw.widgets.IntRangeSlider(
    value=[master_df['Votes Cast'].min(),
           master_df['Votes Cast'].max()],
    min=master_df['Votes Cast'].min(),
    max=master_df['Votes Cast'].max(),
    step=step_finder(acceptable_steps,
                     master_df['Votes Cast'],
                     num_steps=10),
    description='Votes Cast:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format=',d',
    style=style_width_initial,
    layout=layout_large_table_filters_hidden)

In [ ]:
wgt_votes_pct_pop_slider = ipw.widgets.FloatRangeSlider(
    value=[master_df['Votes % of Pop'].min(),
           master_df['Votes % of Pop'].max()],
    min=master_df['Votes % of Pop'].min(),
    max=master_df['Votes % of Pop'].max(),
    step=step_finder(acceptable_steps,
                     master_df['Votes % of Pop'],
                     num_steps=10),
    description='Votes % of Pop:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format=',d',
    style=style_width_initial,
    layout=layout_large_table_filters_hidden)

In [ ]:
wgt_cols_selmult = ipw.widgets.SelectMultiple(
    options=list(master_df.columns),
    value=['County', 'FIPS', 'Year', 'Winning Party', 'Margin Percent',
           'Margin Count', 'Votes Cast', 'Votes % of Pop', 'Population'],
    description='Columns',
    row=10,
    style=style_width_initial,
    layout=layout_tall_table_filters_hidden)

In [ ]:
wgt_download_button = ipw.widgets.Button(
    description='Download',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to download csv of data for current map.',
    icon='download',
    style=style_width_max_content)
#     layout=layout_no_width_hidden)

In [ ]:
wgt_download_status = ipw.widgets.HTML(
    value='',
    description='',
    tooltip='')

In [ ]:
wgt_acs_metric_dropdown = ipw.widgets.Dropdown(
    options=list(acs_metrics_dict.values()),
    value='Population',
    description='Demographic Overlay:',
    style=style_width_initial,
    layout=layout_width_300px)

In [ ]:
# Geojson object to match county FIPS to lat/long
global counties_geojson
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties_geojson = json.load(response)

In [ ]:
# Set aggregation methods for summary table
agg_default_dict = {
    'Margin Count':'median',
    'Margin Percent':'median',
    'Votes Cast':'sum',
    'Votes % of Pop':'median',
    'Population':'sum',
    'Pop %, Male':'median',
    'Pop %, Female':'median',
    'Pop %, Age <18':'median',
    'Pop %, Age 18-24':'median',
    'Pop %, Age 25-34':'median',
    'Pop %, Age 35-44':'median',
    'Pop %, Age 45-54':'median',
    'Pop %, Age 55-64':'median',
    'Pop %, Age 65+':'median',
    'Pop %, White':'median',
    'Pop %, Black':'median',
    'Pop %, N. American & Alaskan':'median',
    'Pop %, Asian':'median',
    'Pop %, Hawaiian & P. Islander':'median',
    'Pop %, Other':'median',
    'Pop %, 2+ Races':'median',
    'Pop %, 9-12th Grade':'median',
    'Pop %, High School':'median',
    'Pop %, Some College No Degree':'median',
    'Pop %, Assoc Degree':'median',
    'Pop %, Bach Degree':'median',
    'Pop %, Graduate Degree':'median',
    'Median Household Income':'median',
    'Pop %, Priv Health Ins':'median',
    'Pop %, No Priv Health Ins':'median',
    'Pop %, Publ Health Ins':'median',
    'Pop %, No Publ Health Ins':'median'}

In [ ]:
# Establish output widget
wgt_output_map = ipw.widgets.Output(
    placeholder='Map will render here.')

wgt_output_graphs = ipw.widgets.Output(
    placeholder='Graphs will render here.')

wgt_output_summary_table = ipw.widgets.Output(
    placeholder='Summary table will render here.')

wgt_output_detailed_table = ipw.widgets.Output(
    placeholder='County detail table will render here.')

In [ ]:
def table_controls_filter(a):
    
    # Clear previous tables
    wgt_output_summary_table.clear_output()
    wgt_output_detailed_table.clear_output()
    
    race_sel = wgt_race_radio.value # 'President', 'US Senate', 'US House', 'NC Senate', 'NC House'
    meth_sel = wgt_method_radio.value # 'Total', 'Early', 'Election Day'
    metr_sel = wgt_calc_dropdown.value # 'Margin Percent', 'Margin Count', 'Votes Cast', 'Votes % of Pop'
    
    # Recreate race_method_filtered_df
    race_method_filtered_df = master_df[(
        master_df['Contest'] == race_sel) & (
        master_df['Method'] == meth_sel)]
    
    # Filter table rows
    race_method_filtered_df = race_method_filtered_df[(
        race_method_filtered_df['Population'] >= wgt_population_slider.value[0]) & (
        race_method_filtered_df['Population'] <= wgt_population_slider.value[1]) & (
        race_method_filtered_df['Year'].isin(wgt_year_selmult.value)) & (
        race_method_filtered_df['Winning Party'].isin(wgt_winning_party_radio.value)) & (
        race_method_filtered_df['Margin Count'] >= wgt_margin_count_slider.value[0]) & (
        race_method_filtered_df['Margin Count'] <= wgt_margin_count_slider.value[1]) & (
        race_method_filtered_df['Margin Percent'] >= wgt_margin_percent_slider.value[0]) & (
        race_method_filtered_df['Margin Percent'] <= wgt_margin_percent_slider.value[1]) & (
        race_method_filtered_df['Votes Cast'] >= wgt_votes_cast_slider.value[0]) & (
        race_method_filtered_df['Votes Cast'] <= wgt_votes_cast_slider.value[1]) & (
        race_method_filtered_df['Votes % of Pop'] >= wgt_votes_pct_pop_slider.value[0]) & (
        race_method_filtered_df['Votes % of Pop'] <= wgt_votes_pct_pop_slider.value[1])]
    
    # Filter table columns, force include (County, Year, and 1+ Numeric Series)
    updated_cols = list(wgt_cols_selmult.value)
    if 'Year' not in wgt_cols_selmult.value:
        updated_cols.insert(0, 'Year')
    if 'County' not in wgt_cols_selmult.value:
        updated_cols.insert(0, 'County')
    numer_cols_chosen = len(
        [value for value in wgt_cols_selmult.value \
         if value in list(agg_default_dict.keys())])
    if numer_cols_chosen == 0:
        updated_cols.append(wgt_calc_dropdown.value)
    race_method_filtered_df = race_method_filtered_df[updated_cols]
    
    # Refresh summary table
    filtered_agg_dict = {
        k:agg_default_dict[k] for k in list(race_method_filtered_df.columns) if k in agg_default_dict}
    summary_df = race_method_filtered_df.groupby('Year').agg(filtered_agg_dict).round(1)
    
    # Apply filters
    with wgt_output_summary_table:
        display(summary_df)
    with wgt_output_detailed_table:
        display(race_method_filtered_df)

## Set widgets to observe update function
## When observe() is enabled, it auto-runs function
wgt_population_slider.observe(table_controls_filter, names='value')
wgt_year_selmult.observe(table_controls_filter, names='value')
wgt_winning_party_radio.observe(table_controls_filter, names='value')
wgt_margin_count_slider.observe(table_controls_filter, names='value')
wgt_margin_percent_slider.observe(table_controls_filter, names='value')
wgt_votes_cast_slider.observe(table_controls_filter, names='value')
wgt_votes_pct_pop_slider.observe(table_controls_filter, names='value')
wgt_cols_selmult.observe(table_controls_filter, names='value')

In [ ]:
def click_map_download_button(btn):
    wgt_download_status.value = '<font color="#FFA500"><i>Downloading...</i></font>'
    race_method_filtered_df.to_csv(
        '{}nc_county_dash_download_{}_{}.csv'.format(
            app_download_path,
            wgt_race_radio.value,
            wgt_method_radio.value),
        index=False)
    os.startfile(app_download_path) # Launch directory in explorer
    wgt_download_status.value = '<font color="green"><i>Complete!</i></font>'
    time.sleep(3)
    wgt_download_status.value = ''
    
# Assign function to button widget
wgt_download_button.on_click(click_map_download_button)

In [ ]:
def click_map_update_button(btn):
    
    try:
        
        # Clear previous exhibits and tables
        wgt_output_map.clear_output()
        wgt_output_graphs.clear_output()
        wgt_output_summary_table.clear_output()
        wgt_output_detailed_table.clear_output()
        wgt_update_progress.layout = layout_visible
        wgt_update_status.value = '<font color="#FFA500"><i>Loading...</i></font>'
        
        # Hide widgets upon refresh
        wgt_bar_header.layout = layout_no_width_hidden
        wgt_summary_table_header.layout = layout_no_width_hidden
        wgt_detailed_table_header.layout = layout_no_width_hidden
        wgt_population_slider.layout = layout_large_table_filters_hidden
        wgt_year_selmult.layout = layout_small_table_filters_hidden
        wgt_winning_party_radio.layout = layout_small_table_filters_hidden
        wgt_margin_count_slider.layout = layout_large_table_filters_hidden
        wgt_margin_percent_slider.layout = layout_large_table_filters_hidden
        wgt_votes_cast_slider.layout = layout_large_table_filters_hidden
        wgt_votes_pct_pop_slider.layout = layout_large_table_filters_hidden
        wgt_cols_selmult.layout = layout_tall_table_filters_hidden
        wgt_selmult_note.layout = layout_small_table_filters_hidden
        
        # Disable filter widgets from observing, so that we can assign
        # new values without triggering table_controls_filter()
        wgt_cols_selmult.unobserve_all(name='value')
        wgt_population_slider.unobserve_all(name='value')
        wgt_year_selmult.unobserve_all(name='value')
        wgt_winning_party_radio.unobserve_all(name='value')
        wgt_margin_count_slider.unobserve_all(name='value')
        wgt_margin_percent_slider.unobserve_all(name='value')
        wgt_votes_cast_slider.unobserve_all(name='value')
        wgt_votes_pct_pop_slider.unobserve_all(name='value')
        wgt_cols_selmult.unobserve_all(name='value')
        
        # Retrieve selections
        race_sel = wgt_race_radio.value # 'President', 'US Senate', 'US House', 'NC Senate', 'NC House'
        meth_sel = wgt_method_radio.value # 'Total', 'Early', 'Election Day'
        metr_sel = wgt_calc_dropdown.value # 'Margin Percent', 'Margin Count', 'Votes Cast', 'Votes % of Pop'
        acs_sel = wgt_acs_metric_dropdown.value # Categories: Population, Gender, Race, Income, etc.
        
        wgt_sel_disp.value = '''
        <b>Selections: <font color="#008080">Race: {}, \
        Method: {}, Metric: {}, Overlay: {}</b></font>'''.format(
            race_sel, meth_sel, metr_sel, acs_sel)
        
        # Update years filter per race (e.g., only show 2016 and 2020 for Presidentail)
        if race_sel == 'President':
            wgt_year_selmult.options = [2016, 2020]
            wgt_year_selmult.value = [2020]
        elif race_sel == 'US Senate':
            wgt_year_selmult.options = [2014, 2016, 2020]
            wgt_year_selmult.value = [2020]
        elif race_sel in ['US House', 'NC Senate', 'NC House']:
            wgt_year_selmult.options = [2014, 2016, 2018, 2020]
            wgt_year_selmult.value = [2020]
        
        # Retrieve df according to widget selections
        global race_method_filtered_df
        race_method_filtered_df = master_df[(
            master_df['Contest'] == race_sel) & (
            master_df['Method'] == meth_sel)]
        map_df = race_method_filtered_df[['County', 'FIPS', 'Year', metr_sel, acs_sel]].copy()
        
        # Update status
        wgt_update_status.value = '<font color="#FFA500"><i>Loading</i></font>'
        wgt_update_progress.value = 1
        
        # Update sliders so lower and upper bounds reflect new table
        wgt_population_slider.value = [race_method_filtered_df['Population'].min(),
                                       race_method_filtered_df['Population'].max(),]
        wgt_population_slider.min = race_method_filtered_df['Population'].min()
        wgt_population_slider.max = race_method_filtered_df['Population'].max()
        wgt_margin_count_slider.value = [race_method_filtered_df['Margin Count'].min(),
                                       race_method_filtered_df['Margin Count'].max(),]
        wgt_margin_count_slider.min = race_method_filtered_df['Margin Count'].min()
        wgt_margin_count_slider.max = race_method_filtered_df['Margin Count'].max()
        wgt_margin_percent_slider.value = [race_method_filtered_df['Margin Percent'].min(),
                                       race_method_filtered_df['Margin Percent'].max(),]
        wgt_margin_percent_slider.min = race_method_filtered_df['Margin Percent'].min()
        wgt_margin_percent_slider.max = race_method_filtered_df['Margin Percent'].max()
        wgt_votes_cast_slider.value = [race_method_filtered_df['Votes Cast'].min(),
                                       race_method_filtered_df['Votes Cast'].max(),]
        wgt_votes_cast_slider.min = race_method_filtered_df['Votes Cast'].min()
        wgt_votes_cast_slider.max = race_method_filtered_df['Votes Cast'].max()
        wgt_votes_pct_pop_slider.value = [race_method_filtered_df['Votes % of Pop'].min(),
                                       race_method_filtered_df['Votes % of Pop'].max(),]
        wgt_votes_pct_pop_slider.min = race_method_filtered_df['Votes % of Pop'].min()
        wgt_votes_pct_pop_slider.max = race_method_filtered_df['Votes % of Pop'].max()
        
        # Set columns filter (on table tab) based on df headers
        wgt_cols_selmult.options = list(race_method_filtered_df.columns)
        wgt_cols_selmult.value = ['County', 'FIPS', 'Year', 'Winning Party', 'Margin Percent',
                                  'Margin Count', 'Votes Cast', 'Votes % of Pop', acs_sel]
        
        # Update status
        wgt_update_status.value = '<font color="#FFA500"><i>Retrieved selections...</i></font>'
        wgt_update_progress.value = 2
        
        # Set color range, with 0 midpoint as purple, neg as red, pos as blue
        min_margin = map_df[metr_sel].min()
        max_margin = map_df[metr_sel].max()
        abs_min = min(abs(min_margin), abs(max_margin))
        
        # Pick colors, range, and midpoint based on method type
        if metr_sel in ['Margin Percent', 'Margin Count']:
            colorscale_choice = ['#FF0000', '#0000FF']
            range_choice = (-abs_min, abs_min)
            midpoint_choice = 0
        else:
            colorscale_choice = 'BuPu'
            range_choice = (0, max_margin)
            midpoint_choice = max_margin / 2
        
        # Create map
        map_fig = px.choropleth_mapbox(
            map_df,
            geojson=counties_geojson,
            animation_frame='Year',
            locations='FIPS',
            hover_data =[metr_sel, acs_sel],
            hover_name=map_df['County'].str.title(),
            color=metr_sel,
            color_continuous_scale=colorscale_choice,
            color_continuous_midpoint=0,
            range_color=range_choice,
            mapbox_style='carto-positron',
            zoom=6,
            center = {'lat': 35.540699, 'lon': -79.774559}, # Seagrove, NC centerpoint
            opacity=0.5)
        map_fig.update_layout(margin={'r':0,'t':0,'l':0,'b':0})
        map_fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 2000
        map_fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 300
        
        # Update status
        wgt_update_status.value = '<font color="#FFA500"><i>Map complete...</i></font>'
        wgt_update_progress.value = 3
        
        # Create bar chart for election metric
        pop_min_df = race_method_filtered_df[race_method_filtered_df['Population'] >= 100000]
        bar_elec_df = pop_min_df[['County', metr_sel, 'FIPS', 'Year']].copy()
        bar_elec_df['Color'] = np.where(
            bar_elec_df[metr_sel] > 0,
            'Positive', 'Negative')
        bar_elec_df.sort_values(['Year', 'County'], ascending=True, inplace=True)
        if bar_elec_df[metr_sel].min() < 0:
            smart_y_range = [bar_elec_df[metr_sel].min()*1.05,
                             bar_elec_df[metr_sel].max()*1.05]
        elif bar_elec_df[metr_sel].min() > 0:
            smart_y_range = [bar_elec_df[metr_sel].min()*0.95,
                             bar_elec_df[metr_sel].max()*1.05]
        else:
            smart_y_range = [0, bar_elec_df[metr_sel].max()*1.05]
        bar_elec_fig = px.bar(
            bar_elec_df,
            x=bar_elec_df['County'].str.title(),
            y=metr_sel,
            color='Color',
            color_discrete_map={
                'Negative':'#FF0000',
                'Positive':'#0000FF'},
            labels={'x':''},
            range_x=[0, bar_elec_df['Year'].value_counts().max()],
            range_y=smart_y_range,
            animation_frame='Year',
            animation_group='County')
        bar_elec_fig.update_layout(
            coloraxis_showscale=False,
            showlegend=False)
        bar_elec_fig.update_xaxes(categoryorder='category ascending')
        bar_elec_fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 1000
        bar_elec_fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 300
        
        # Update status
        wgt_update_status.value = '<font color="#FFA500"><i>Election results graph complete...</i></font>'
        wgt_update_progress.value = 3
        
        # Create bar chart for ACS metric
        bar_acs_df = pop_min_df[['County', acs_sel, 'FIPS', 'Year']].copy()
        bar_acs_df.sort_values(['Year', acs_sel, 'County'], ascending=True, inplace=True)
        if bar_acs_df[acs_sel].min() < 0:
            smart_y_range = [bar_acs_df[acs_sel].min()*1.05,
                             bar_acs_df[acs_sel].max()*1.05]
        elif bar_acs_df[acs_sel].min() > 0:
            smart_y_range = [bar_acs_df[acs_sel].min()*0.95,
                             bar_acs_df[acs_sel].max()*1.05]
        else:
            smart_y_range = [0, bar_acs_df[acs_sel].max()*1.05]
        bar_acs_fig = px.bar(
            bar_acs_df,
            x=bar_acs_df['County'].str.title(),
            y=acs_sel,
            color_continuous_scale='BuPu',
            labels={'x':''},
            range_x=[0, bar_acs_df['Year'].value_counts().max()],
            range_y=smart_y_range,
            animation_frame='Year',
            animation_group='County')
        bar_acs_fig.update_layout(
            coloraxis_showscale=False,
            showlegend=False)
        bar_acs_fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 1000
        bar_acs_fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 300
        
        # Update status
        wgt_update_status.value = '<font color="#FFA500"><i>Demographics graph complete...</i></font>'
        wgt_update_progress.value = 4
        
        # Filter table columns
        race_method_filtered_df = race_method_filtered_df[list(wgt_cols_selmult.value)]
        
        # Create summary table
        filtered_agg_dict = {
            k:agg_default_dict[k] for k in list(race_method_filtered_df.columns) if k in agg_default_dict}
        summary_df = race_method_filtered_df.groupby('Year').agg(filtered_agg_dict).round(1)
        
        # Update status
        wgt_update_status.value = '<font color="#FFA500"><i>Data tables complete...</i></font>'
        wgt_update_progress.value = 5
        
        # Render map and table in respective tabs
        with wgt_output_map:
            map_fig.show()
        with wgt_output_graphs:
            bar_elec_fig.show()
            bar_acs_fig.show()
        with wgt_output_summary_table:
            display(summary_df)
        with wgt_output_detailed_table:
            display(race_method_filtered_df)
        
        # Update status
        wgt_update_status.value = '<font color="#FFA500"><i>Dashboard updating...</i></font>'
        wgt_update_progress.value = 6
        
        # Show various widgets, now that maps, exhibits, and table are rendered
        wgt_bar_header.layout = layout_no_width_visible
        wgt_summary_table_header.layout = layout_no_width_visible
        wgt_detailed_table_header.layout = layout_no_width_visible
        wgt_population_slider.layout = layout_large_table_filters_visible
        wgt_year_selmult.layout = layout_small_table_filters_visible
        wgt_winning_party_radio.layout = layout_small_table_filters_visible
        wgt_margin_count_slider.layout = layout_large_table_filters_visible
        wgt_margin_percent_slider.layout = layout_large_table_filters_visible
        wgt_votes_cast_slider.layout = layout_large_table_filters_visible
        wgt_votes_pct_pop_slider.layout = layout_large_table_filters_visible
        wgt_cols_selmult.layout = layout_tall_table_filters_visible
        wgt_selmult_note.layout = layout_small_table_filters_visible
        
        # Reobserve filter widgets
        wgt_population_slider.observe(table_controls_filter, names='value')
        wgt_year_selmult.observe(table_controls_filter, names='value')
        wgt_winning_party_radio.observe(table_controls_filter, names='value')
        wgt_margin_count_slider.observe(table_controls_filter, names='value')
        wgt_margin_percent_slider.observe(table_controls_filter, names='value')
        wgt_votes_cast_slider.observe(table_controls_filter, names='value')
        wgt_votes_pct_pop_slider.observe(table_controls_filter, names='value')
        wgt_cols_selmult.observe(table_controls_filter, names='value')
        
        # Update status
        wgt_update_status.value = '<font color="#green"><i>Complete!</i></font>'
        wgt_update_progress.value = 7
        time.sleep(3)
        wgt_update_status.value = ''
        wgt_update_progress.layout = layout_hidden
    
    except KeyError as e:
        wgt_update_status.value = 'Error'
        wgt_sel_disp.value = '''
        <b><font color="#FF0000">Error, please change selection</b></font>'''
        with wgt_output_map:
            display(e)

# Assign function to button widget
wgt_update_button.on_click(click_map_update_button)

In [ ]:
# Dashboard controls
headers = ipw.HBox([wgt_app_desc,
                    wgt_extra_space,
                    wgt_help])
update_button_status = ipw.HBox(
    [wgt_update_button,
     wgt_update_progress,
     wgt_update_status])
download_button_status = ipw.HBox(
    [wgt_download_button,
     wgt_download_status])
control_panel = ipw.VBox(
    [ipw.HBox([wgt_calc_dropdown, wgt_acs_metric_dropdown]),
     ipw.HBox([wgt_race_radio,
               wgt_method_radio])])
buttons_box = ipw.VBox(
    [update_button_status, download_button_status])
controls_box = ipw.VBox(
    [headers, control_panel, buttons_box, wgt_sel_disp])

# Exhibits and graphs
exhibits_box = ipw.VBox([wgt_output_map,
                         wgt_extra_space,
                         wgt_bar_header,
                         wgt_output_graphs])

# Tables
table_controls = ipw.HBox([
    ipw.VBox([wgt_selmult_note,
              wgt_year_selmult,
              wgt_cols_selmult]),
    ipw.VBox([wgt_winning_party_radio,
              wgt_population_slider,
              wgt_margin_count_slider,
              wgt_margin_percent_slider,
              wgt_votes_cast_slider,
              wgt_votes_pct_pop_slider])])
table_box = ipw.VBox([table_controls,
                      wgt_extra_space,
                      wgt_summary_table_header,
                      wgt_output_summary_table,
                      wgt_extra_space,
                      wgt_detailed_table_header,
                      wgt_output_detailed_table])

# Tabs: (1) map and graphs, (2) tables
wgt_tab = ipw.widgets.Tab()
wgt_tab.children = [exhibits_box, table_box]
wgt_tab.set_title(0, 'Exhibits')
wgt_tab.set_title(1, 'Table')

# Store controls and tabs in single container
app_container = ipw.VBox(
    [controls_box, wgt_tab])

In [ ]:
app_container